# Feature Backfill for Google Trends
**Goal of this notebook**

This notebook will backfill the feature groups containing google trends data and the flight data
* Supports backfill
* Produces daily features
* Is point-in-time correct
* Uploads features to Hopsworks Feature Store

**Imports & setup**

In [27]:
# Core
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import asyncio

# Google Trends
from pytrends.request import TrendReq

# Hopsworks
import hopsworks


## Google Trends Feature Backfilll

### Feature Config

In [21]:
# Search terms used as predictors
KEYWORDS = [
    "vikings",
    "fika",
    "stockholm",
    "ikea",
    "abba"
]

# Country code for Sweden
COUNTRY = "SE"

# Rolling windows (days)
ROLLING_WINDOWS = [7, 14]

# Google Trends data delay (days)
DATA_DELAY = 1

### Pipeline mode and data logic (backfill vs daily)

In [22]:
# Pipeline mode: "backfill" or "daily"
PIPELINE_MODE = "daily"

# Backfill range (used only in backfill mode)
BACKFILL_START_DATE = date(2023, 1, 1)
BACKFILL_END_DATE   = date(2024, 12, 31)

# Resolve execution dates
today = date.today()

if PIPELINE_MODE == "backfill":
    start_date = BACKFILL_START_DATE
    end_date   = BACKFILL_END_DATE
else:
    end_date   = today - timedelta(days=DATA_DELAY)
    start_date = end_date

print(f"Running feature pipeline from {start_date} to {end_date}")

Running feature pipeline from 2025-12-28 to 2025-12-28


### Fetch Google Trends Data

In [23]:
def fetch_google_trends(keywords, start_date, end_date):
    """
    Fetch Google Trends data for given keywords and date range.
    Returns weekly data indexed by date.
    """
    pytrends = TrendReq(hl="en-US", tz=360)

    timeframe = f"{start_date} {end_date}"

    pytrends.build_payload(
        kw_list=keywords,
        timeframe=timeframe,
        geo=COUNTRY
    )

    df = pytrends.interest_over_time()
    return df

### Clean and resample to Daily Data

In [24]:
# Fetch raw data
raw_trends = fetch_google_trends(KEYWORDS, start_date, end_date)

# Remove partial rows (important!)
raw_trends = raw_trends[raw_trends["isPartial"] == False]

# Drop metadata column
raw_trends = raw_trends.drop(columns=["isPartial"])

# Convert to daily frequency using forward-fill
daily_trends = (
    raw_trends
    .resample("D")
    .ffill()
    .reset_index()
)

# Rename date column
daily_trends = daily_trends.rename(columns={"date": "event_date"})
daily_trends["event_date"] = pd.to_datetime(daily_trends["event_date"]).dt.date

daily_trends.head()

TooManyRequestsError: The request failed: Google returned a response with code 429

### Feature Engineering 

In [ ]:
features_df = daily_trends.copy()

for kw in KEYWORDS:
    # Rolling averages
    for window in ROLLING_WINDOWS:
        features_df[f"{kw}_{window}d_avg"] = (
            features_df[kw]
            .rolling(window=window, min_periods=1)
            .mean()
        )

    # Weekly change
    features_df[f"{kw}_7d_delta"] = (
        features_df[kw] - features_df[kw].shift(7)
    )

features_df.head()

### Connect to Hopsworks

In [7]:
project = hopsworks.login()
fs = project.get_feature_store()

2025-12-29 16:49:47,106 INFO: Initializing external client
2025-12-29 16:49:47,107 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-12-29 16:49:48,989 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1296539


### Create/get Feature Group

In [9]:
feature_group = fs.get_or_create_feature_group(
    name="google_trends_daily",
    version=1,
    primary_key=["event_date"],
    description="Daily Google Trends features for Sweden tourism prediction",
    online_enabled=False
)

### Write features to Hopsworks

In [ ]:
feature_group.insert(
    features_df,
    write_options={"wait_for_job": True}
)